In [2]:
import rasterio as rio
import cfar_filters as cfar
import numpy as np
from pathlib import Path

In [3]:
# Replace these paths with the actual paths to your TIFF files
hh_image_path = Path('sentinel1-image-HH.tiff')
hv_image_path = Path('sentinel1-image-HV.tiff')

# Open and read the HH polarization image
with rio.open(hh_image_path) as src:
    hh_image = src.read(1)  # read(1) reads the first band

# Open and read the HV polarization image
with rio.open(hv_image_path) as src:
    hv_image = src.read(1)  # read(1) reads the first band

# Check if the images have the same shape
if hh_image.shape != hv_image.shape:
    raise ValueError("The images do not have the same shape!")

# Stack the images to create a single array of shape (2, X, Y)
in_image = np.array([hh_image, hv_image], dtype=np.float32)

print(f"Combined array shape: {in_image.shape}")

Combined array shape: (2, 16393, 16748)


In [3]:
image_filename = Path('sentinel1-image-HH.tiff')
with rio.open(image_filename) as src:
    in_image = src.read()

print(in_image.shape)

(1, 16393, 16748)


In [ ]:
in_image = np.squeeze(in_image)
print(in_image.shape)

In [ ]:
# load your SAR image as a numpy array
# image format should be a numpy.ndarray(float32) of shape (2,X,Y) with the order: HH, HV
'''
image_filename = Path('sentinel1-image-HH.tiff')
with rio.open(image_filename) as src:
    in_image = src.read()
'''

#in_image = in_image[1]
# define mask - could also include landmask
mask = np.nansum(in_image, axis=0)!=0
print(mask.shape)
in_image = in_image[0, :, :]
print(in_image.shape)

pfa = 1e-15  # probability of false alarm rate
enl = 10.7  # equivalent number of looks of the image
wi = 9  # diameter of the guard area
wo = 15  # diameter of the clutter estimation area

# using the gamma detector
#gamma_outliers = cfar.gamma.detector(in_image, mask, pfa, enl, wi, wo)

# using the lognormal detector
db_image = cfar.utils.in2db(in_image)  # convert image to decibel using .utils
lognormal_outliers = cfar.lognormal.detector(db_image, mask, pfa, wi, wo)

# using the k detector
N = 40  # number of steps for the LUT
#k_outliers = cfar.kdistribution.detector(in_image, mask, N, pfa, enl, wi, wo)

# remove objects smaller than 3 using .utils
#gamma_outliers = cfar.utils.remove_small_objects(gamma_outliers, 2)

(16393, 16748)
(16393, 16748)


C:\cfar-object-detection\.venv\lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
C:\cfar-object-detection\.venv\lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
C:\cfar-object-detection\cfar_filters\lognormal.py:59: UserWarning: Input image should be in decibel scale. Image smells like fishy
  warnings.warn(f'Input image should be in decibel scale. Image smells like {smells_like(image)}',
C:\cfar-object-detection\cfar_filters\lognormal.py:62: RuntimeWarning: invalid value encountered in subtract
  image = (image - np.nanmean(image)) / np.nanstd(image)  # standardize the data
C:\cfar-object-detection\.venv\lib\site-packages\numpy\lib\nanfunctions.py:1741: RuntimeWarning: invalid value encountered in subtract
  np.subtract(arr, avg, out=arr, casting='unsafe', where=where)
